In [1]:
# Import from the globals_and_helpers file
from globals_and_helpers import (
    PROJECT_DIR,
    MODELS_DIR,
    ORIG_IMAGE_DIR,
    TEMP_OUTPUTS_DIR,
    FINAL_OUTPUTS_DIR,
    TILE_SIZE,
    BASE_NAMES,
    normalize_image,
    plot_histogram,
    get_dapi_and_collagen_paths,
    convert_seconds_to_hms,
    get_base_name,
    check_image_dimensions,
    gen_STIFMap_tile_path,
)

In [2]:
from STIFMaps import STIFMap_generation
from STIFMaps.misc import get_step

import os
import re
import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from scipy import interpolate
from PIL import Image
import tifffile
import time

# STIFMap models
models = [
    os.path.join(MODELS_DIR, 'iteration_1171.pt'),
    os.path.join(MODELS_DIR, 'iteration_1000.pt'),
    os.path.join(MODELS_DIR, 'iteration_1043.pt'),
    os.path.join(MODELS_DIR, 'iteration_1161.pt'),
    os.path.join(MODELS_DIR, 'iteration_1180.pt')
]

# Parameters
# STIFMap_SCALE_FACTOR = 0.5
STIFMap_SCALE_FACTOR = 2.712
STIFMap_STEP = get_step(40, STIFMap_SCALE_FACTOR)
STIFMap_SQUARE_SIDE = get_step(224, STIFMap_SCALE_FACTOR)
STIFMap_BATCH_SIZE = 100

print('Step size is ' + str(STIFMap_STEP) + ' pixels')
print('Side length for a square is ' + str(STIFMap_SQUARE_SIDE) + ' pixels')

Step size is 14 pixels
Side length for a square is 82 pixels


In [3]:
# Function to generate and save the STIFMap
def gen_STIFMap_tile(dapi_path, collagen_path, name, step, models, batch_size, square_side, check_existing=True):
    output_path = gen_STIFMap_tile_path(dapi_path)
    
    # Check if the tile has already been processed
    if check_existing and os.path.exists(output_path):
        print(f"Skipping already processed tile: {dapi_path} and {collagen_path}")
        return

    start_time = time.perf_counter()

    z_out = STIFMap_generation.generate_STIFMap(
        dapi=dapi_path, 
        collagen=collagen_path, 
        name=name, 
        step=step, 
        models=models,
        mask=False, 
        batch_size=batch_size, 
        square_side=square_side,
        save_dir=False
    )

    end_time = time.perf_counter()
    print("Elapsed time:", convert_seconds_to_hms(end_time - start_time))

    output_image = np.mean(z_out, axis=0)

    global_min = np.min(output_image)
    global_max = np.max(output_image)
    output_image_normalized = (output_image - global_min) / (global_max - global_min)

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    plt.imsave(output_path, output_image_normalized, cmap="viridis")
    print(f"Saved image: {output_path}")

    # Save the raw stiffness values as a NumPy array
    stiffness_values_path = output_path.replace(".png", ".npy")
    np.save(stiffness_values_path, output_image)
    print(f"Saved stiffness values: {stiffness_values_path}")

# Function to check if a tile has already been processed
def is_tile_completed(output_path):
    return os.path.exists(output_path)

# dapi_path, collagen_path = get_dapi_and_collagen_paths(
#     base_name=BASE_NAMES[0], orig_image_dir=ORIG_IMAGE_DIR
# )
# print(f"dapi_path={dapi_path}, collagen_path={collagen_path}")
# gen_STIFMap_tile(
#     # dapi_path=dapi_path,
#     # collagen_path=collagen_path,
#     dapi_path='/home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/27620/IPMN_tiles/27620_C0_0_0.tif',
#     collagen_path='/home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/27620/IPMN_tiles/27620_C1_0_0.tif',
#     name='test',
#     step=STIFMap_STEP,
#     models=models,
#     batch_size=STIFMap_BATCH_SIZE,
#     square_side=STIFMap_SQUARE_SIDE,
#     check_existing=True  # Set to True to check if the file already exists
# )

In [4]:
def stitch_images(output_filename, base_name, image_format='png'):
    # Dynamically determine the number of rows and columns
    tile_pattern = re.compile(rf"{base_name}_(\d+)_(\d+)\.{image_format}")
    STIFMaps_directory = os.path.join(TEMP_OUTPUTS_DIR, base_name, "STIFMap_tiles")
    
    row_col_map = {}
    for file in os.listdir(STIFMaps_directory):
        match = tile_pattern.match(file)
        if match:
            row, col = map(int, match.groups())
            row_col_map.setdefault(row, set()).add(col)

    if not row_col_map:
        raise ValueError("No matching files found in the directory.")

    # Correctly determine the number of rows and columns
    max_row = max(row_col_map.keys())
    max_col = max(max(cols) for cols in row_col_map.values())
    num_rows = max_row + 1
    num_cols = max_col + 1

    print(f"Detected grid size: {num_rows} rows x {num_cols} columns")

    # Determine image dimensions from the first available .png tile
    image_width = image_height = None
    for row in range(num_rows):
        for col in range(num_cols):
            image_filename = f"{base_name}_{row}_{col}.{image_format}"
            image_path = os.path.join(STIFMaps_directory, image_filename)
            if os.path.exists(image_path):
                try:
                    with Image.open(image_path) as image:
                        image_width, image_height = image.size
                    break
                except Exception as e:
                    print(f"Error opening {image_path}: {e}")
        if image_width is not None:
            break

    if image_width is None:
        raise ValueError("No valid .png image files found to determine dimensions.")

    stitched_width = num_cols * image_width
    stitched_height = num_rows * image_height
    stitched_image = Image.new('RGB', (stitched_width, stitched_height), color='white')

    for row in range(num_rows):
        for col in range(num_cols):
            image_filename = f"{base_name}_{row}_{col}.{image_format}"
            image_path = os.path.join(STIFMaps_directory, image_filename)

            if os.path.exists(image_path):
                try:
                    image = Image.open(image_path)
                except Exception as e:
                    print(f"Error opening {image_path}: {e}")
                    image = Image.new('RGB', (image_width, image_height), color='white')
                    print(f"Missing tile: {image_filename}. Replacing with a white tile.")
            else:
                image = Image.new('RGB', (image_width, image_height), color='white')
                print(f"Missing tile: {image_filename}. Replacing with a white tile.")

            x = col * image_width
            y = row * image_height
            stitched_image.paste(image, (x, y))

    stitched_image.save(output_filename)
    print(f"Stitched image saved as {output_filename}")

In [5]:
def get_base_file_name(file_path):
    # Extract the file name without the extension
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    # Keep only the first two parts (e.g., '27620_C0') by splitting on '_'
    base_name = "_".join(file_name.split("_")[:2])
    return base_name
    
# Generate the STIFMaps for all the tiles for a base image
def gen_STIFMap(base_name):
    dapi_path, collagen_path = get_dapi_and_collagen_paths(base_name, orig_image_dir=ORIG_IMAGE_DIR)
    base_name_C0 = get_base_file_name(dapi_path)
    base_name_C1 = get_base_file_name(collagen_path)    
    
    # Automatically determine num_rows and num_cols based on filenames
    tile_pattern_C0 = re.compile(rf"{base_name_C0}_(\d+)_(\d+)\.tif")
    tile_pattern_C1 = re.compile(rf"{base_name_C1}_(\d+)_(\d+)\.tif")

    # tile_pattern = re.compile(rf"{base_name_C0}_(\d+)_(\d+)\.tif")
    tile_image_dir = os.path.join(TEMP_OUTPUTS_DIR, base_name, "IPMN_tiles")
    
    # Dictionary to store how many columns exist per row
    row_col_map = {}
    
    # Loop through files and match patterns
    for file in os.listdir(tile_image_dir):
        # match = tile_pattern.match(file)
        match_C0 = tile_pattern_C0.match(file)
        match_C1 = tile_pattern_C1.match(file)

        if match_C0 or match_C1:
            match = match_C0 or match_C1
            row, col = map(int, match.groups())
            row_col_map.setdefault(row, set()).add(col)
            # print(f"Matched Row: {row}, Column: {col}")
    
    # Find the maximum number of columns
    if not row_col_map:
        raise ValueError("No matching files found in the directory.")
    
    max_cols = max(len(cols) for cols in row_col_map.values())
    
    # Select only rows with the full set of columns
    valid_rows = sorted([row for row, cols in row_col_map.items() if len(cols) == max_cols])
    num_rows = len(valid_rows)
    num_cols = max_cols

    print("Row to Columns mapping:", row_col_map)
    print("Valid rows with full columns:", valid_rows)
    
    print(f"Detected grid size: {num_rows} rows x {num_cols} columns")
    
    # Create a list of file names for C0 and C1 tiles
    # C0_files = [f"{base_name_C0}_{i}_{j}.tif" for i in range(num_rows) for j in range(num_cols)]
    # C1_files = [f"{base_name_C1}_{i}_{j}.tif" for i in range(num_rows) for j in range(num_cols)]
    
    # Main: Loop through all the tiled C0 and C1 images and pass each one to run_STIFMap()
    for row in valid_rows:  # Iterate only over rows with full columns
        for col in range(num_cols):
            dapi_path = os.path.join(tile_image_dir, f"{base_name_C0}_{row}_{col}.tif")
            collagen_path = os.path.join(tile_image_dir, f"{base_name_C1}_{row}_{col}.tif")
    
            # Ensure files exist before processing
            if not os.path.exists(dapi_path) or not os.path.exists(collagen_path):
                print(f"Skipping missing tile: {dapi_path} or {collagen_path}")
                continue
            
            output_path = gen_STIFMap_tile_path(dapi_path)
            # output_path = gen_output_path(dapi_path)
            
            if is_tile_completed(output_path):
                print(f"Skipping already processed tile: {dapi_path} and {collagen_path}")
                continue

            print(f"Generating STIFMap for {dapi_path}, {collagen_path}")
            gen_STIFMap_tile(
                dapi_path, collagen_path, name=base_name, step=STIFMap_STEP,
                models=models, batch_size=STIFMap_BATCH_SIZE, square_side=STIFMap_SQUARE_SIDE,
                check_existing=True
            )
            
# gen_STIFMap(base_name=BASE_NAMES[1])

In [ ]:
for base_name in BASE_NAMES:
    gen_STIFMap(base_name=base_name)

Row to Columns mapping: {1: {0, 1, 2, 3, 4, 5, 6, 7}, 5: {0, 1, 2, 3, 4, 5, 6, 7}, 3: {0, 1, 2, 3, 4, 5, 6, 7}, 4: {0, 1, 2, 3, 4, 5, 6, 7}, 2: {0, 1, 2, 3, 4, 5, 6, 7}, 0: {0, 1, 2, 3, 4, 5, 6, 7}}
Valid rows with full columns: [0, 1, 2, 3, 4, 5]
Detected grid size: 6 rows x 8 columns
Generating STIFMap for /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/4601/IPMN_tiles/4601_C0_0_0.tif, /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/4601/IPMN_tiles/4601_C1_0_0.tif
Device is cpu


/home/steve/.local/lib/python3.10/site-packages/STIFMaps/STIFMap_generation.py:145: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  io.imshow(np.moveaxis(im, 0, -1))


Image shape is (3, 5003, 5003)
Num squares in x direction is 352
Num squares in y direction is 352
Time taken to predict squares is 484.21207761764526
Time taken to predict squares is 961.0957415103912
Time taken to predict squares is 1437.8494093418121
Time taken to predict squares is 1916.2542178630829
Time taken to predict squares is 2399.234855413437
Total time taken is 2400.24493432045
Elapsed time: 40 minutes, 0.3 seconds
Saved image: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/4601/STIFMap_tiles/4601_0_0.png
Saved stiffness values: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/4601/STIFMap_tiles/4601_0_0.npy
Generating STIFMap for /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/4601/IPMN_tiles/4601_C0_0_1.tif, /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/4601/IPMN_tiles/4601_C1_0_1.tif
Device is cpu


/home/steve/.local/lib/python3.10/site-packages/STIFMaps/STIFMap_generation.py:145: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  io.imshow(np.moveaxis(im, 0, -1))


Image shape is (3, 5003, 5003)
Num squares in x direction is 352
Num squares in y direction is 352
Time taken to predict squares is 498.47581148147583
Time taken to predict squares is 996.6242170333862
Time taken to predict squares is 1492.3138995170593
Time taken to predict squares is 1996.2228999137878
Time taken to predict squares is 2519.0575902462006
Total time taken is 2520.034378051758
Elapsed time: 42 minutes, 0.1 seconds
Saved image: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/4601/STIFMap_tiles/4601_0_1.png
Saved stiffness values: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/4601/STIFMap_tiles/4601_0_1.npy
Generating STIFMap for /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/4601/IPMN_tiles/4601_C0_0_2.tif, /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/4601/IPMN_tiles/4601_C1_0_2.tif
Device is cpu


/home/steve/.local/lib/python3.10/site-packages/STIFMaps/STIFMap_generation.py:145: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  io.imshow(np.moveaxis(im, 0, -1))


Image shape is (3, 5003, 5003)
Num squares in x direction is 352
Num squares in y direction is 352


In [43]:
# Stitch the processed tiles back together
# stitched_output_path = os.path.join(STIFMaps_directory, f"{base_name[0]}_STIFMap_stitched_v2.png")
# base_name = BASE_NAMES[0]
# stitched_output_path = os.path.join(TEMP_DIR, f"{base_name}_STIFMap_stitched.png")
# stitch_images(stitched_output_path, base_name, image_format='png')

Detected grid size: 15 rows x 18 columns
Missing tile: 27620_STIFMap_13_3.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_4.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_5.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_6.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_7.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_8.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_9.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_10.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_11.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_12.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_13.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_14.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_15.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_16.png. Replacing with a white tile.
Missing tile: 